In [39]:
import os
from nustar_gen import info
ns = info.NuSTAR()
import numpy as np
from astropy.time import Time
from astropy.io import fits

In [40]:
caldb = os.getenv('CALDB')
gain_path = caldb+'/data/nustar/fpm/bcf/gain/'
now = Time.now()

In [42]:
# vcit011 failed during the "intermediate" plateau (found a bug)
# vcit012 should fix the bug
# vcit013 allowed FPMA DET3 to have a single linear slope
# vcit014 fits for FPMA DET2 gain drift in epoch 1

for mod in ['A', 'B']:

    # Read in the output from the Parse_Chain_Fits notebook
    gain_data = np.loadtxt(f'gain_fit_{mod}_v2.txt')

    # Load the base gain file from the CALDB
    gain_base = gain_path+f'nu{mod}gain20100101v007.fits'
    new_gain = f'nu{mod}gain20100101vcit014.fits'
    
    with fits.open(gain_base) as base_hdu:
        base_hdu.info()
        new_hdu_list = fits.HDUList(base_hdu[0])
        for det in range(4):
#             if mod is 'A' and det == 2:
#                 continue
            
            base_gain_change = gain_data[det*3][2]
            base_gain_change = 1.0
            gain_change1 = gain_data[det*3+1][1]
            slope_change1 = gain_data[det*3+1][2]
            gain_change2 = gain_data[det*3+2][1]
            slope_change2 = gain_data[det*3+2][2]


            
            data = base_hdu[det+1].data
            hdr = base_hdu[det+1].header
            hdr['DATE'] = now.isot
            
            # Find all of the rows associated with the 2015 gain change:
            base_time_mask = data['TIME'] == data['TIME'][0]
            base_time_ind = np.where(data['TIME'] == data['TIME'][0])[0]

            nrows_base = data.shape[0]
            base_data = data[base_time_mask]
            base_data['SLOPE'] *= base_gain_change
            temp_hdu = fits.BinTableHDU(data=base_data)
            nrows_base = data[base_time_mask].shape[0]


            
            cal_time_2015 = data['TIME'][1]
            
                        
            first_change = temp_hdu.copy()
            second_change = temp_hdu.copy()
            third_change = temp_hdu.copy()


            # Include this bit to force the change for FPMA DET2A for the 2015 epoch:
            
            first_change.data['SLOPE'] /= slope_change1
            first_change.data['TIME'] = [cal_time_2015 for x in second_change.data['TIME']]


            # Second change is flat, so just use the same slopes as the first one
            second_change.data['TIME'] = [gain_change1 for x in second_change.data['TIME']]
            second_change.data['SLOPE'] /= slope_change1
            
            # Third change has the slope dropping again.
            third_change.data['TIME'] = [gain_change2 for x in third_change.data['TIME']]
            third_change.data['SLOPE'] /= slope_change2 
            
            nrows_first = first_change.data.shape[0]
            nrows_second = second_change.data.shape[0]
            nrows_third = third_change.data.shape[0]

            # Extend old HDU to have the new number of rows
            base_hdu[det+1] = fits.BinTableHDU.from_columns(base_hdu[det+1].columns,
                                                    nrows = nrows_base + nrows_first + nrows_second + nrows_third)

            # Copy over header information
            base_hdu[det+1].header = hdr
            
            # Copy over all of the gain data
            for colname in base_hdu[det+1].columns.names:
                base_hdu[det+1].data[colname][0:nrows_base] = temp_hdu.data[colname]
                base_hdu[det+1].data[colname][nrows_base:nrows_base+nrows_first] = first_change.data[colname]
                base_hdu[det+1].data[colname][nrows_base+nrows_first:nrows_base+nrows_second+nrows_first] = second_change.data[colname]
                base_hdu[det+1].data[colname][(nrows_base+nrows_first+nrows_second):] = third_change.data[colname]
    
        # Confirm that things worked okay
        base_hdu.info()
        
        # Write output to new gain file.
        base_hdu.writeto(new_gain, overwrite=True)
        
        print()

Filename: /Users/bwgref/science/local/CALDB/data/nustar/fpm/bcf/gain/nuAgain20100101v007.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      12   ()      
  1  GAIN          1 BinTableHDU     58   2048R x 6C   [D, B, B, 3E, 3E, 3E]   
  2  GAIN          2 BinTableHDU     58   2048R x 6C   [D, B, B, 3E, 3E, 3E]   
  3  GAIN          3 BinTableHDU     58   2048R x 6C   [D, B, B, 3E, 3E, 3E]   
  4  GAIN          4 BinTableHDU     58   2048R x 6C   [D, B, B, 3E, 3E, 3E]   
Filename: /Users/bwgref/science/local/CALDB/data/nustar/fpm/bcf/gain/nuAgain20100101v007.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      12   ()      
  1  GAIN          1 BinTableHDU     58   4096R x 6C   ['D', 'B', 'B', '3E', '3E', '3E']   
  2  GAIN          2 BinTableHDU     58   4096R x 6C   ['D', 'B', 'B', '3E', '3E', '3E']   
  3  GAIN          3 BinTableHDU     58   4096R x 6C   ['D', 'B', 'B', '3E', '3E',

array([0.3351 , 0.33319, 0.33275], dtype=float32)